### **DEMO 1A**

In [ ]:
from docplex.cp.model import *
import docplex.cp.utils_visu as visu

#### TRES PROCESOS SIN RESTRICCIONES

In [ ]:
m = CpoModel()

p1 = m.interval_var(name='PROCESO 1', size=10)
p2 = m.interval_var(name='PROCESO 2', size=20)
p3 = m.interval_var(name='PROCESO 3', size=15)

m.add(p1, p2, p3) # agregar los intervalos

s = m.solve()

s.print_solution()

if s:
    # Gráfico de Gantt
    visu.interval(s.get_var_solution(p1), 1, 'P1')  
    visu.interval(s.get_var_solution(p2), 2, 'P2')  
    visu.interval(s.get_var_solution(p3), 3, 'P3')        

visu.show()

#### TRES PROCESOS PERO SOLO DOS MÁQUINAS

In [ ]:
m = CpoModel()

p1 = m.interval_var(name='PROCESO 1', size=10)
p2 = m.interval_var(name='PROCESO 2', size=20)
p3 = m.interval_var(name='PROCESO 3', size=15)

m.add(m.pulse(p1,1)+m.pulse(p2,1)+m.pulse(p3,1) <= 2)   # Restricción de recursos (máquinas)

s = m.solve()

s.print_solution()

 
if s:
    p1s = s.get_var_solution(p1)
    p2s = s.get_var_solution(p2)
    p3s = s.get_var_solution(p3)

    # Gráfico de Gantt
    visu.panel(name="Gantt")
    visu.interval(p1s, 1, 'P1')  
    visu.interval(p2s, 2, 'P2')  
    visu.interval(p3s, 3, 'P3')  

    maq = CpoStepFunction()
    maq.add_value(p1s.start, p1s.end, 1)
    maq.add_value(p2s.start, p2s.end, 1)
    maq.add_value(p3s.start, p3s.end, 1)

    # Gráfico de utilización de recursos
    visu.panel(name="Uso de recursos")
    visu.function(segments=maq, style='area', origin=0, horizon=30, color='khaki')     

visu.show()

### **DEMO 1B**

#### LIMITACIÓN DE MATERIA PRIMA USANDO STEP_AT_START

In [ ]:
m = CpoModel()

t1 = m.interval_var(optional=True, name='PROCESO 1', size=10)   # muy importante optional = True
t2 = m.interval_var(optional=True, name='PROCESO 2', size=10)
t3 = m.interval_var(optional=True, name='PROCESO 3', size=10)

m.add(m.step_at_start(t1,28)+m.step_at_start(t2,12)+m.step_at_start(t3,10) <= 45)   # Restricción de materia prima

m.add(m.end_before_start(t1,t2))

m.add(m.maximize(m.presence_of(t1)+m.presence_of(t2)+m.presence_of(t3)))

s = m.solve()

s.print_solution()

if s:    
    t1s = s.get_var_solution(t1)
    t2s = s.get_var_solution(t2)
    t3s = s.get_var_solution(t3)

    # Gráfico de Gantt
    visu.panel(name="Gantt")
    if t1s.is_present(): visu.interval(t1s, 1, 'P1')  
    if t2s.is_present(): visu.interval(t2s, 2, 'P2')  
    if t3s.is_present(): visu.interval(t3s, 3, 'P3')  

    max_int = 1000
    maq = CpoStepFunction()
    if t1s.is_present(): maq.add_value(t1s.start, max_int, 28)
    if t2s.is_present(): maq.add_value(t2s.start, max_int, 12)
    if t3s.is_present(): maq.add_value(t3s.start, max_int, 10)

    # Gráfico de utilización de recursos
    visu.panel(name="Consumo de MP")
    visu.function(segments=maq, style='area', origin = -1, horizon = 30, color='lightgreen')     
    

visu.show()

### **DEMO 1C**

#### EJEMPLO DE USO DE STEP_AT_END

In [ ]:
from docplex.cp.model import CpoModel, CpoStepFunction
import docplex.cp.utils_visu as visu

m = CpoModel()

a1 = m.interval_var(size = 5, name = 'A1')
a2 = m.interval_var(size = 5, name = 'A2')
a3 = m.interval_var(size = 5, name = 'A3')
pk = m.interval_var(size = 2, name = 'PK')

m.add(m.pulse(a1,1)+m.pulse(a2,1)+m.pulse(a3,1)<=1) # un operario

f = m.step_at_end(a1,1) + m.step_at_end(a2,1) + m.step_at_end(a3,1) # funcion que acumula produccion

m.add(m.always_in(f, pk, 3, 3))

s = m.solve()

s.print_solution()

if s:    
    a1s = s.get_var_solution(a1)
    a2s = s.get_var_solution(a2)
    a3s = s.get_var_solution(a3)
    pks = s.get_var_solution(pk)

    # Gráfico de Gantt
    visu.panel(name="Gantt")
    if a1s.is_present(): visu.interval(a1s, 'coral', 'A1')  
    if a2s.is_present(): visu.interval(a2s, 'lightgreen', 'A2')  
    if a3s.is_present(): visu.interval(a3s, 'lightblue', 'A3')  
    if pks.is_present(): visu.interval(pks, 'yellow', 'PK')  
    
    max_int = 1000
    maq = CpoStepFunction()
    if a1s.is_present(): maq.add_value(a1s.end, max_int, 1)
    if a2s.is_present(): maq.add_value(a2s.end, max_int, 1) # += step_at
    if a3s.is_present(): maq.add_value(a3s.end, max_int, 1)
    if pks.is_present(): maq.add_value(pks.end, max_int, -3)
    

    # Gráfico de utilización de recursos
    visu.panel(name="Productos")
    visu.function(segments=maq, style='area', origin=0, horizon=30, color='salmon')     

visu.show()

### **DEMO 1D**

### CALENDARIO PARA EL MES DE DICIEMBRE

In [ ]:
from docplex.cp.model import CpoModel, CpoStepFunction
import docplex.cp.utils_visu as visu

dic = CpoStepFunction()

dic.set_value(0, 31, 100)       # carga todos al 100%

for i in range(0, 31, 7):       # fines de semana
    dic.set_value(i, i+2, 0)

for i in [8, 25]:               # feriados
    dic.set_value(i, i+1, 0)

m = CpoModel()

a1 = m.interval_var(name='A1', size = 5, intensity=dic)
a2 = m.interval_var(name='A2', size = 4, intensity=dic)
a3 = m.interval_var(name='A3', size = 3, intensity=dic)

m.add(m.end_before_start(a1,a2))
m.add(m.end_before_start(a2,a3))

s = m.solve()

s.print_solution()

if s:
    visu.panel()
    visu.pause(dic) # marca los no laborables
    visu.interval(s.get_var_solution(a1), 1, 'A1')
    visu.interval(s.get_var_solution(a2), 2, 'A2')
    visu.interval(s.get_var_solution(a3), 3, 'A3')
    visu.show(origin=0, horizon=30)

### USO DE FORBID

In [ ]:
from docplex.cp.model import CpoModel, CpoStepFunction
import docplex.cp.utils_visu as visu

dic = CpoStepFunction()

dic.set_value(0, 31, 100)       # carga todos al 100%

for i in range(0, 31, 7):       # fines de semana
    dic.set_value(i, i+2, 0)

for i in [8, 25]:               # feriados
    dic.set_value(i, i+1, 0)

m = CpoModel()

a1 = m.interval_var(name='A1', size = 5, intensity=dic)
a2 = m.interval_var(name='A2', size = 4, intensity=dic)
a3 = m.interval_var(name='A3', size = 3, intensity=dic)

m.add(m.end_before_start(a1,a2))
m.add(m.end_before_start(a2,a3))

#m.add(m.forbid_start(a1, dic))
#m.add(m.forbid_end(a1, dic))
m.add(m.forbid_extent(a1, dic))

#m.add(m.forbid_start(a2, dic))
#m.add(m.forbid_end(a2, dic))
m.add(m.forbid_extent(a2, dic))

#m.add(m.forbid_start(a3, dic))
#m.add(m.forbid_end(a3, dic))
m.add(m.forbid_extent(a3, dic))

s = m.solve()

s.print_solution()

if s:
    visu.panel()
    visu.pause(dic)
    visu.interval(s.get_var_solution(a1), 1, 'A1')
    visu.interval(s.get_var_solution(a2), 2, 'A2')
    visu.interval(s.get_var_solution(a3), 3, 'A3')
    visu.show(origin=0, horizon=30)